# Metadata

**L1 Taxonomy** - Backend Development

**L2 Taxonomy** - API Gateways

**Subtopic** - Adding request transformation (URI rewrite or header addition) in a Python API Gateway

**Use Case** - Develop a Python script that simulates an API Gateway. The script should be capable of receiving HTTP requests, rewriting the URI, and adding custom headers before forwarding the request to the intended destination. The destination should be a mock endpoint within the same script. The script should also handle the response from the mock endpoint, removing any added headers before sending the response back to the original requester.

**Programming Language** - Python

**Target Model** - GPT-4o

# Setup

```requirements.txt
```


# Prompt
Problem Statement:
- Develop a Python script that simulates an API Gateway.
- The script should be capable of receiving HTTP requests, rewriting the URI, and adding custom headers before forwarding the request to the intended destination.
- The destination should be a mock endpoint within the same script.
- The script should also handle the response from the mock endpoint, removing any added headers before sending the response back to the original requester.

Input format:
- A HTTP `GET` or `POST` request sent to the endpoint `/api/<resource>`.
- The request may include query parameters (for GET) or a JSON body (for POST).
- Optional custom headers may be included in the request.

Input constraints:
- The URI path must begin with `/api/` followed by a valid resource name (only alphanumeric characters, no slashes or special characters).
- For `GET` requests, query parameters (if any) must be URL-encoded.
- For `POST` requests, the body must be valid JSON and not exceed 10KB in size.
- Custom headers must follow standard HTTP header naming conventions (e.g., `X-Request-ID`, `X-Trace-ID`).
- The number of headers (including custom ones) must not exceed 20(the script must enforce this manually).

Output format:
- The response returned to the original requester will be the result from the mock endpoint.
- All custom headers added by the gateway will be stripped before sending the response.
- The response body will be in JSON format.
- The response will include a status code (`200 OK` for success, appropriate error codes otherwise).
- Example response:
```python
  {
    "resource": "data",
    "status": "received",
    "timestamp": "2025-07-30T12:00:00Z"
  }
```

Class Definition and Function Signature:
```python
- class APIGatewayHandler(http.server.BaseHTTPRequestHandler)
    - def do_GET(self) -> None
    - def do_POST(self) -> None
    - def forward_request(self, method: str, path: str, headers: dict, body: bytes) -> tuple[int, dict, bytes]
    - def rewrite_uri(self, path: str) -> str
    - def add_custom_headers(self, headers: dict) -> dict
    - def remove_custom_headers(self, headers: dict) -> dict
    - def mock_endpoint(self, path: str, method: str, headers: dict, body: bytes) -> tuple[int, dict, bytes]
```

Example:
```python
Request:
GET /api/data?item=42 HTTP/1.1
Host: localhost:8080
User-Agent: test-client
X-Request-ID: req-001

Response:
HTTP/1.1 200 OK
Content-Type: application/json

{
  "resource": "data",
  "status": "received",
  "item": "42",
  "timestamp": "2025-07-30T12:00:00Z"
}
```

# Requirements
Explicit Requirements:
- The script must run an HTTP server that accepts `GET` and `POST` requests at `/api/<resource>`.
- It must rewrite the incoming URI before forwarding the request.
- It must add custom headers before forwarding the request to a mock endpoint.
- It must forward the request to a mock endpoint within the same script.
- It must remove any added custom headers from the response before returning it to the client.
- The response must be returned in JSON format.
- The mock response should include a timestamp field in ISO 8601 format.
- Timestamps in the mock response must follow the ISO 8601 format: YYYY-MM-DDTHH:MM:SSZ (UTC)

Implicit Requirements:
- The script must correctly parse HTTP requests and handle routing logic.
- The rewritten URI and custom headers must not break the request semantics.
- Header modification must not interfere with standard HTTP headers.
- The mock endpoint should simulate realistic processing behavior.
- The server must use only standard Python libraries (e.g., `http.server`, `json`, etc.).

Solution Expectations:
- The solution must be implemented in a single Python script.
- It must use the built-in `http.server` module to create the HTTP server.
- The `APIGatewayHandler` class should extend `http.server.BaseHTTPRequestHandler`.
- URI rewriting logic should transform `/api/<resource>` to `/internal/<resource>`.
- Custom headers (e.g., `X-Gateway-ID`) should be added before forwarding the request.
- The `mock_endpoint` method should simulate a response based on the rewritten URI and request contents.
- The response from the mock endpoint must have added headers removed before sending it back to the original client.
- All responses must be well-formed JSON with appropriate status codes.
- The server should support concurrent requests using `http.server.ThreadingHTTPServer`.
- Resource name must match the regex: ^[a-zA-Z0-9]+$

Edge Cases and Behaviour:
- If the request URI does not start with `/api/`, respond with `404 Not Found`.
- If a `POST` request contains invalid JSON, respond with `400 Bad Request`.
- If the request method is not `GET` or `POST`, respond with `405 Method Not Allowed`.
- If required headers are missing, continue processing using default values without crashing.
- If the request body exceeds 10KB, respond with `413 Payload Too Large`.
- If the internal mock endpoint raises an exception, respond with `500 Internal Server Error`.
- Ensure custom headers (e.g., `X-Gateway-ID`) are not leaked in the final response.
- Always return a JSON response, even in error cases, to maintain consistency.

Solution Constraints:
- The script must be implemented using only Python’s standard library (no third-party packages).
- The server must run on localhost and listen on a configurable port (default 8080).
- The total size of the request body must not exceed 10KB.
- Custom headers added by the gateway must use the `X-` prefix and follow standard HTTP naming conventions.
- URI rewriting must strictly convert `/api/<resource>` to `/internal/<resource>` without altering query parameters.
- The mock endpoint logic must reside within the same script and not call external services.
- The server must handle both `GET` and `POST` methods explicitly; all others must return `405 Method Not Allowed`.
- Responses must be in valid JSON format with proper `Content-Type: application/json` headers.
- Error handling must not expose internal stack traces or implementation details to the client.

In [ ]:
# code
# code
"""API Gateway Simulator using Python's http.server and socketserver.

This server simulates API gateway behavior:
- Validates URI path and request body
- Limits headers and body size
- Adds/removes custom headers
- Forwards to a mock internal endpoint
"""

import http.server
import socketserver
import json
import re
from urllib.parse import urlparse, parse_qs
from datetime import datetime, timezone

PORT = 8080
MAX_BODY_SIZE = 10 * 1024  # 10KB
MAX_HEADERS = 20

GATEWAY_CUSTOM_HEADERS = {
    "X-Gateway-ID": "api-gateway-001",
    "X-Trace-ID": "trace-12345",
}


class APIGatewayHandler(http.server.BaseHTTPRequestHandler):
    """Handles HTTP requests as a simulated API Gateway."""

    def do_GET(self) -> None:
        """Handle GET requests."""
        self.handle_request("GET")

    def do_POST(self) -> None:
        """Handle POST requests."""
        self.handle_request("POST")

    def handle_request(self, method: str) -> None:
        """Parse and validate incoming request, then forward to mock."""
        parsed_path = urlparse(self.path)
        path = parsed_path.path

        if not path.startswith("/api/"):
            self.send_error_response(404, "Not Found", "Invalid API path.")
            return

        resource_match = re.match(r"/api/([a-zA-Z0-9]+)$", path)
        if not resource_match:
            self.send_error_response(
                404, "Not Found", "Invalid resource name in API path."
            )
            return

        body = b""
        if method == "POST":
            content_length = int(self.headers.get("Content-Length", 0))
            if content_length > MAX_BODY_SIZE:
                self.send_error_response(
                    413,
                    "Payload Too Large",
                    f"Request body exceeds {MAX_BODY_SIZE} bytes.",
                )
                return
            try:
                body = self.rfile.read(content_length)
                json.loads(body)  # Validate JSON
            except json.JSONDecodeError:
                self.send_error_response(400,
                                         "Bad Request", "Invalid JSON body.")
                return
            except Exception:
                self.send_error_response(
                    400, "Bad Request", "Error reading request body."
                )
                return

        forward_headers = {k: v for k, v in self.headers.items()}

        if len(forward_headers) + len(GATEWAY_CUSTOM_HEADERS) > MAX_HEADERS:
            self.send_error_response(
                400,
                "Bad Request",
                f"Too many headers. Maximum allowed is {MAX_HEADERS}.",
            )
            return

        try:
            (status_code, response_headers,
             response_body) = self.forward_request(
                method, path, forward_headers, body
            )
            self.send_response(status_code)
            for header, value in response_headers.items():
                self.send_header(header, value)
            self.end_headers()
            self.wfile.write(response_body)
        except Exception as e:
            print(f"Error during request forwarding: {e}")
            self.send_error_response(
                500, "Internal Server Error", "An unexpected error occurred."
            )

    def forward_request(
        self, method: str, path: str, headers: dict, body: bytes
    ) -> tuple[int, dict, bytes]:
        """Forward the request to a mock internal endpoint."""
        rewritten_path = self.rewrite_uri(path)
        modified_headers = self.add_custom_headers(headers)
        (status_code, mock_response_headers,
         mock_response_body) = self.mock_endpoint(
            rewritten_path, method, modified_headers, body
        )
        final_response_headers = (self.remove_custom_headers
                                  (mock_response_headers))
        return status_code, final_response_headers, mock_response_body

    def rewrite_uri(self, path: str) -> str:
        """Rewrite the incoming URI to an internal path."""
        parsed_path = urlparse(path)
        resource_name = parsed_path.path.split("/api/", 1)[1]
        new_path = f"/internal/{resource_name}"
        if parsed_path.query:
            new_path += f"?{parsed_path.query}"
        return new_path

    def add_custom_headers(self, headers: dict) -> dict:
        """Add gateway-specific custom headers."""
        new_headers = headers.copy()
        new_headers.update(GATEWAY_CUSTOM_HEADERS)
        return new_headers

    def remove_custom_headers(self, headers: dict) -> dict:
        """Remove gateway-injected headers from the response."""
        final_headers = headers.copy()
        for header_name in GATEWAY_CUSTOM_HEADERS:
            final_headers.pop(header_name, None)
        return final_headers

    def mock_endpoint(
        self, path: str, method: str, headers: dict, body: bytes
    ) -> tuple[int, dict, bytes]:
        """Simulate a mock internal endpoint and return dummy response."""
        parsed_path = urlparse(path)
        resource_name = parsed_path.path.split("/internal/", 1)[1]
        query_params = parse_qs(parsed_path.query)

        response_payload = {
            "resource": resource_name,
            "status": "received",
            "timestamp": datetime.now(timezone.utc)
            .isoformat(timespec="seconds")
            .replace("+00:00", "Z"),
        }

        if method == "GET":
            for key, values in query_params.items():
                response_payload[key] = values[0]
        elif method == "POST" and body:
            try:
                request_data = json.loads(body)
                response_payload.update(request_data)
            except json.JSONDecodeError:
                pass

        mock_headers = {
            "Content-Type": "application/json",
            "X-Internal-Processing-Time": "10ms",
            "X-Gateway-ID": "should-be-removed",
            "X-Trace-ID": "should-also-be-removed",
        }
        mock_headers.update(
            {
                k: v
                for k, v in headers.items()
                if k.startswith("X-") and k not in GATEWAY_CUSTOM_HEADERS
            }
        )

        return 200, mock_headers, json.dumps(response_payload).encode("utf-8")

    def send_error_response(
        self, status_code: int, status_message: str, error_detail: str
    ) -> None:
        """Send JSON-formatted error response."""
        self.send_response(status_code)
        self.send_header("Content-Type", "application/json")
        self.end_headers()
        response = {
            "status": "error",
            "code": status_code,
            "message": status_message,
            "details": error_detail,
            "timestamp": datetime.now(timezone.utc)
            .isoformat(timespec="seconds")
            .replace("+00:00", "Z"),
        }
        self.wfile.write(json.dumps(response).encode("utf-8"))


def run_server():
    """Start the threaded HTTP server."""
    with (socketserver.ThreadingTCPServer(("", PORT),
                                          APIGatewayHandler) as httpd):
        print(f"Serving on port {PORT}")
        try:
            httpd.serve_forever()
        except KeyboardInterrupt:
            print("\nShutting down the server.")
            httpd.shutdown()
            httpd.server_close()


if __name__ == "__main__":
    run_server()


In [ ]:
# tests

"""Test suite for API Gateway Handler functionality."""

import unittest
import json
import threading
import time
import socket
from http.client import HTTPConnection
from main import APIGatewayHandler
import socketserver


class ReuseAddrTCPServer(socketserver.TCPServer):
    """TCPServer that enables socket reuse for Docker environments."""

    allow_reuse_address = True

    def server_bind(self):
        """Configure socket for address reuse."""
        self.socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        super().server_bind()


class TestAPIGateway(unittest.TestCase):
    """Test suite for API Gateway functionality."""

    @classmethod
    def setUpClass(cls):
        """Start the test server once for all tests."""
        cls.server_port = 8001

        for attempt in range(3):
            try:
                cls.server = ReuseAddrTCPServer(
                    ("", cls.server_port), APIGatewayHandler
                )
                break
            except OSError as e:
                if e.errno == 98 and attempt < 2:
                    time.sleep(1)
                    continue
                raise

        cls.server_thread = threading.Thread(target=cls.server.serve_forever)
        cls.server_thread.daemon = True
        cls.server_thread.start()
        time.sleep(0.5)

    @classmethod
    def tearDownClass(cls):
        """Stop the test server."""
        try:
            cls.server.shutdown()
            cls.server.server_close()
            time.sleep(0.1)
        except Exception:
            pass

    def setUp(self):
        """Set up HTTP connection for each test."""
        self.conn = HTTPConnection("localhost", self.server_port)

    def tearDown(self):
        """Close HTTP connection after each test."""
        self.conn.close()

    def _make_request(self, method, path, headers=None, body=None):
        """Make HTTP requests."""
        if headers is None:
            headers = {}

        if body is not None and isinstance(body, dict):
            body = json.dumps(body)
            if "Content-Type" not in headers:
                headers["Content-Type"] = "application/json"

        self.conn.request(method, path, body, headers)
        response = self.conn.getresponse()
        response_body = response.read().decode("utf-8")

        return {
            "status": response.status,
            "headers": dict(response.headers),
            "body": json.loads(response_body) if response_body else None,
        }

    def test_get_request_success(self):
        """Test successful GET request with path rewriting."""
        headers = {"X-Client-ID": "test-client-123"}
        response = self._make_request("GET", "/gateway/api/users", headers)

        self.assertEqual(response["status"], 200)
        self.assertEqual(response["body"]["path"], "/mock/api/users")
        self.assertEqual(response["body"]["method"], "GET")
        self.assertEqual(response["body"]["client_id"], "test-client-123")
        self.assertEqual(response["body"]["status"], "success")

    def test_post_request_with_json_body(self):
        """Test POST request with valid JSON body."""
        headers = {
            "X-Client-ID": "test-client",
            "Content-Type": "application/json",
        }
        body = {"message": "hello", "user": "john"}
        response = self._make_request(
            "POST", "/gateway/api/resource", headers, body
        )

        self.assertEqual(response["status"], 200)
        self.assertEqual(response["body"]["path"], "/mock/api/resource")
        self.assertEqual(response["body"]["method"], "POST")
        self.assertEqual(response["body"]["client_id"], "test-client")
        self.assertEqual(response["body"]["received_data"], body)

    def test_put_request_with_headers(self):
        """Test PUT request with JSON body and custom headers."""
        headers = {
            "Content-Type": "application/json",
            "X-Client-ID": "client-456",
        }
        body = {"name": "Updated User", "age": 30}
        response = self._make_request(
            "PUT", "/gateway/api/users/123", headers, body
        )

        self.assertEqual(response["status"], 200)
        self.assertEqual(response["body"]["path"], "/mock/api/users/123")
        self.assertEqual(response["body"]["method"], "PUT")
        self.assertEqual(response["body"]["client_id"], "client-456")
        self.assertEqual(response["body"]["received_data"], body)

    def test_delete_request(self):
        """Test DELETE request handling."""
        headers = {"X-Client-ID": "admin-client"}
        response = self._make_request(
            "DELETE", "/gateway/api/users/456", headers
        )

        self.assertEqual(response["status"], 200)
        self.assertEqual(response["body"]["path"], "/mock/api/users/456")
        self.assertEqual(response["body"]["method"], "DELETE")
        self.assertEqual(response["body"]["client_id"], "admin-client")

    def test_invalid_path_not_gateway(self):
        """Test request to path not starting with /gateway/."""
        response = self._make_request("GET", "/api/users")

        self.assertEqual(response["status"], 404)
        self.assertEqual(response["body"]["error"], "Not found")

    def test_invalid_method_patch(self):
        """Test unsupported HTTP method (PATCH)."""
        response = self._make_request("PATCH", "/gateway/api/resource")

        self.assertEqual(response["status"], 405)
        self.assertEqual(response["body"]["error"], "Method not allowed")

    def test_invalid_json_body(self):
        """Test POST with malformed JSON."""
        headers = {"Content-Type": "application/json"}
        self.conn.request(
            "POST", "/gateway/api/data", "{invalid json}", headers
        )
        response = self.conn.getresponse()
        response_body = json.loads(response.read().decode("utf-8"))

        self.assertEqual(response.status, 400)
        self.assertEqual(response_body["error"], "Invalid JSON payload")

    def test_oversized_payload(self):
        """Test request body exceeding 10KB limit."""
        headers = {"Content-Type": "application/json"}
        large_data = {"data": "x" * 11000}
        response = self._make_request(
            "POST", "/gateway/api/upload", headers, large_data
        )

        self.assertEqual(response["status"], 413)
        self.assertEqual(response["body"]["error"], "Payload too large")

    def test_get_with_query_parameters(self):
        """Test GET request with query string preservation."""
        response = self._make_request(
            "GET", "/gateway/api/search?q=test&limit=10&page=2"
        )

        self.assertEqual(response["status"], 200)
        self.assertEqual(
            response["body"]["path"], "/mock/api/search?q=test&limit=10&page=2"
        )
        self.assertEqual(response["body"]["method"], "GET")

    def test_gateway_headers_removed(self):
        """Verify gateway-specific headers are removed from response."""
        headers = {"X-Client-ID": "test-verify-headers"}
        response = self._make_request("GET", "/gateway/api/headers", headers)

        self.assertEqual(response["status"], 200)
        self.assertNotIn("X-Gateway-Processed", response["headers"])
        self.assertNotIn("X-Forwarded-For", response["headers"])

    def test_post_without_content_type(self):
        """Test POST request missing Content-Type header."""
        self.conn.request("POST", "/gateway/api/data", '{"test": "data"}')
        response = self.conn.getresponse()
        response_body = json.loads(response.read().decode("utf-8"))

        self.assertEqual(response.status, 400)
        self.assertEqual(
            response_body["error"],
            "Content-Type required for POST/PUT requests",
        )

    def test_empty_body_post(self):
        """Test POST with empty body."""
        headers = {"Content-Type": "application/json"}
        response = self._make_request(
            "POST", "/gateway/api/empty", headers, ""
        )

        self.assertEqual(response["status"], 400)
        self.assertEqual(
            response["body"]["error"], "Empty body not allowed for POST/PUT"
        )


if __name__ == "__main__":
    unittest.main(verbosity=2)


# Model Breaking Proof

#### Model Breaking Task URL: https://rlhf-v3.turing.com/prompt/39a8dfba-93b0-4e19-96a1-9da69e9d465e?origin=https%3A%2F%2Flabeling-ms.turing.com&redirect_url=https%3A%2F%2Flabeling-ms.turing.com%2Fconversations%2F83960%2Fview
#### Model code:

```python
# code generated by the model
import http.server
import json
import socketserver
from urllib.parse import urlparse, parse_qs
from datetime import datetime

class APIGatewayHandler(http.server.BaseHTTPRequestHandler):
    def do_GET(self):
        self.handle_request('GET')

    def do_POST(self):
        self.handle_request('POST')

    def handle_request(self, method):
        # Check the request path
        if not self.path.startswith('/api/'):
            self.respond_with_error(404, "Not Found")
            return

        # Validate header count
        if len(self.headers) > 20:
            self.respond_with_error(400, "Too Many Headers")
            return

        # Parse path and query
        parsed_url = urlparse(self.path)
        # <Issue>: Resource name does not match the regex: ^[a-zA-Z0-9]+$
        resource = parsed_url.path.split('/')[2]  # Extract <resource>
        
        # Validate resource
        if not resource.isalnum():
            self.respond_with_error(400, "Invalid Resource Name")
            return

        headers = dict(self.headers)
        if method == "POST":
            content_length = int(self.headers.get('Content-Length', '0'))
            if content_length > 10 * 1024:  # 10KB
                self.respond_with_error(413, "Payload Too Large")
                return

            try:
                body = self.rfile.read(content_length)
                json_body = json.loads(body)  # Validate JSON
            except json.JSONDecodeError:
                self.respond_with_error(400, "Bad Request: Invalid JSON")
                return
        else:
            json_body = None  # No body for GET

        rewritten_uri = self.rewrite_uri(parsed_url.path)
        new_headers = self.add_custom_headers(headers)
        
        # Forward request to mock endpoint
        status_code, response_headers, response_body = self.forward_request(
            method, rewritten_uri, new_headers, json.dumps(json_body).encode() if json_body else b""
        )
        
        response_headers = self.remove_custom_headers(response_headers)
        self.respond_with_body(status_code, response_headers, response_body)

    def rewrite_uri(self, path: str) -> str:
        # Convert /api/<resource> to /internal/<resource>, maintain query parameters
        return path.replace('/api/', '/internal/', 1)

    def add_custom_headers(self, headers: dict) -> dict:
        headers['X-Gateway-ID'] = 'gateway-1234'
        return headers
    
    def remove_custom_headers(self, headers: dict) -> dict:
        headers.pop('X-Gateway-ID', None)
        return headers

    def forward_request(self, method: str, path: str, headers: dict, body: bytes) -> tuple:
        try:
            return self.mock_endpoint(path, method, headers, body)
        except Exception as e:
            self.respond_with_error(500, "Internal Server Error")
            return 500, {}, b"" # <Issue>: May return error twice.

    def mock_endpoint(self, path: str, method: str, headers: dict, body: bytes) -> tuple:
        # Simulate a mock endpoint which processes data
        parsed_path = urlparse(path)
        resource_name = parsed_path.path.split('/')[2]
        query_components = parse_qs(parsed_path.query)

        response_data = {
            "resource": resource_name,
            "status": "received",
            "timestamp": datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')
        }

        # Add query params to response for illustration (GET-specific)
        if method == 'GET':
            response_data.update({k: v[0] for k, v in query_components.items()})

        # For POST, we could include some behavior based on 'body' if required

        response_body = json.dumps(response_data).encode('utf-8')
        return 200, {'Content-Type': 'application/json'}, response_body

    def respond_with_body(self, status: int, headers: dict, body: bytes):
        self.send_response(status)
        for key, value in headers.items():
            self.send_header(key, value)
        self.end_headers()
        self.wfile.write(body)

    def respond_with_error(self, status, message):
        error_body = json.dumps({'error': message}).encode('utf-8')
        self.respond_with_body(status, {'Content-Type': 'application/json'}, error_body)

if __name__ == "__main__":
    PORT = 8080
    # <Issue>: The server does not support concurrent requests using http.server.ThreadingHTTPServer.
    with socketserver.TCPServer(("", PORT), APIGatewayHandler) as httpd:
        print(f"Serving at port {PORT}")
        httpd.serve_forever()
```